In [12]:
import pandas as pd
import numpy as np

In [1]:
%%bash
source ~/.bashrc
conda activate assembly
cd /storage/group/ibb3/default/sanger_reannotation/funannotate_annotation/annotate_results/
busco \
        -m genome \
        -c 8 \
        -i Acropora_palmata.scaffolds.fa \
        -o BUSCO \
        -f \
        -l metazoa_odb10 \
        --download_path ~/scratch/

2024-07-19 17:13:49 INFO:	***** Start a BUSCO v5.4.3 analysis, current time: 07/19/2024 17:13:49 *****
2024-07-19 17:13:49 INFO:	Configuring BUSCO with local environment
2024-07-19 17:13:49 INFO:	Mode is genome
2024-07-19 17:13:49 INFO:	Downloading information on latest versions of BUSCO data...
2024-07-19 17:13:52 INFO:	Input file is /storage/group/ibb3/default/sanger_reannotation/funannotate_annotation/annotate_results/Acropora_palmata.scaffolds.fa
2024-07-19 17:13:52 INFO:	Downloading file 'https://busco-data.ezlab.org/v5/data/lineages/metazoa_odb10.2024-01-08.tar.gz'
2024-07-19 17:13:57 INFO:	Decompressing file '/storage/home/nsl5160/scratch/lineages/metazoa_odb10.tar.gz'
2024-07-19 17:14:07 INFO:	Running BUSCO using lineage dataset metazoa_odb10 (eukaryota, 2024-01-08)
2024-07-19 17:14:07 INFO:	Running 1 job(s) on bbtools, starting at 07/19/2024 17:14:07
2024-07-19 17:14:11 INFO:	[bbtools]	1 of 1 task(s) completed
2024-07-19 17:14:13 INFO:	Running 1 job(s) on metaeuk, starting at 

In [ ]:
!cat ../linkage_map/apal_linkage.map | awk -F'\t' -vOFS='\t' '{ $1 = "" $1 ".1" }1' \
    > ../funannotate_annotation/annotate_results/Acropora_palmata_corrected_headers.map

In [ ]:
!cat ../funannotate_annotation/annotate_results/Acropora_palmata.gff3 \
    | awk -F'\t' -vOFS='\t' '/^[^#]/ { $1 = "" $1 ".1" }1' \
    > ../funannotate_annotation/annotate_results/Acropora_palmata_corrected_headers.gff3

In [ ]:
!sed '/^>/s/$/.1/' ../funannotate_annotation/annotate_results/Acropora_palmata.scaffolds.fa \
    > ../funannotate_annotation/annotate_results/Acropora_palmata_corrected_headers.fa

In [ ]:
!awk -F'\t' -vOFS='\t' '{ $4 = "" $4 ".1" }1' ../funannotate_annotation/annotate_results/Acropora_palmata.annotations_proteinfer.txt \
    > ../funannotate_annotation/annotate_results/Acropora_palmata_annotation_corrected_headers.txt

In [ ]:
%%bash
source ~/.bashrc
conda activate assembly
gff2bed < ../repeatmasked/GCA_964030605.1_jaAcrPala1.1_genomic.fa.out.gff \
    > ../funannotate_annotation/annotate_results/Acropora_palmata_repeatmasked_corrected_headers.bed

In [18]:
%%bash
source ~/.bashrc
conda activate assembly
cd ../funannotate_annotation/annotate_results/
bedtools merge -i Acropora_palmata_repeatmasked_corrected_headers.bed > bookended_repeats.bed
paste <( cat bookended_repeats.bed ) \
    <( cat bookended_repeats.bed | cut -f1-2 | sed "s/\t/_/g" ) \
    > bookended_repeats_renamed.bed
bedtools intersect -wo -a Acropora_palmata_corrected_headers.gff3 \
    -b bookended_repeats_renamed.bed -bed | awk '$3~"gene"' \
    | bedtools groupby -g 1-9 -c 14 -o sum \
    > Acropora_palmata_tmp.gff3

In [26]:
repeats = pd.read_csv("../funannotate_annotation/annotate_results/Acropora_palmata_tmp.gff3", 
            sep = "\t", header = None)
repeats["repeat_overlap"] = repeats[9] / (repeats[4] - repeats[3] + 1)

In [32]:
repeats[[0, 1, 2, 3, 4, 5, 6, 7, 8]][repeats["repeat_overlap"] >= 0.9].to_csv("../funannotate_annotation/annotate_results/repeat_genes.gff3", 
                                                 sep = "\t", header = None, index = False)

In [37]:
%%bash
source ~/.bashrc
conda activate assembly
cd ../funannotate_annotation/annotate_results/
gff2bed < repeat_genes.gff3 | cut -f1-4 > Acropora_palmata_repeat_genes.bed

In [39]:
%%bash
cp ../funannotate_annotation/annotate_results/Acropora_palmata_repeat_genes.bed \
    ../../apal_acer_genomes/sanger_references/Apalmata.repetitive_genes.bed
cp ../funannotate_annotation/annotate_results/Acropora_palmata_corrected_headers.gff3 \
    ../../apal_acer_genomes/sanger_references/Apalmata.gff3
cp ../funannotate_annotation/annotate_results/Acropora_palmata_repeatmasked_corrected_headers.bed \
    ../../apal_acer_genomes/sanger_references/Apalmata.repeatmask.bed
cp ../funannotate_annotation/annotate_results/Acropora_palmata_annotation_corrected_headers.txt \
    ../../apal_acer_genomes/sanger_references/Apalmata.annotations.txt
cp ../funannotate_annotation/annotate_results/Acropora_palmata_corrected_headers.fa \
    ../../apal_acer_genomes/sanger_references/Apalmata.fa
cp ../funannotate_annotation/annotate_results/Acropora_palmata_corrected_headers.map \
    ../../apal_acer_genomes/sanger_references/Apalmata_SEXAVG.map